In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import trim

# Crear una sesión de Spark
spark = SparkSession.builder \
    .appName("Preprocessing Data") \
    .getOrCreate()

# Cargar datos desde un archivo CSV
oij_df = spark.read.csv("C:/Users/grana/OneDrive/Escritorio/Bases de datos/spark/data/OIJ.csv", header=True, inferSchema=True)
inec_df = spark.read.csv("C:/Users/grana/OneDrive/Escritorio/Bases de datos/spark/data/INEC.csv", header=True, inferSchema=True)

def eliminar_espacios(dataframe, columna):
    """
    Elimina los espacios en blanco al inicio y final de los valores en una columna específica.

    Args:
        dataframe (DataFrame): DataFrame de entrada (PySpark).
        columna (str): Nombre de la columna a procesar.

    Returns:
        DataFrame: DataFrame con la columna modificada.
    """
    return dataframe.withColumn(columna, trim(dataframe[columna]))

# Eliminar espacios en blanco en la columna 'Distrito' en ambos DataFrames
oij_df = eliminar_espacios(oij_df, 'Distrito')
inec_df = eliminar_espacios(inec_df, 'Provincia cantón y distrito')  # Ajustar si la columna tiene otro nombre

# Mostrar los resultados
print("Datos del OIJ después de limpiar:")
oij_df.show(50, truncate=False)

print("Datos del INEC después de limpiar:")
inec_df.show(50, truncate=False)

